In [ ]:
# default_exp retrieve

In [ ]:
import config
import requests
import base64
import json
import pandas as pd

# retrieval

> Pull all tracks and their features for a user's public playlists

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
with open('spot_tokens.json') as file:
    tokens = json.load(file)

In [ ]:
access_token = tokens['access_token']
refresh_token = tokens['refresh_token']

In [ ]:
df_items = pd.DataFrame()

user_id = 'atraylor6'
offset = 0

while len(df_items) % 50 == 0:
    
    # print(len(df_items))

    play_url = f'https://api.spotify.com/v1/users/{user_id}/playlists?limit=50&offset={offset}'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }

    r_play = requests.get(play_url, headers=headers)
    

    if r_play.status_code == 401:

        TOKEN_URL = 'https://accounts.spotify.com/api/token'

        message = config.CLIENT_ID + ':' + config.CLIENT_SECRET
        messageBytes = message.encode('ascii')
        base64Bytes = base64.b64encode(messageBytes)
        base64Message = base64Bytes.decode('ascii')
    
        headers = {
        'Authorization': 'Basic ' + base64Message,
        'Content-Type': 'application/x-www-form-urlencoded'
        }

        pars_refresh = {
            'grant_type': 'refresh_token',
            'refresh_token': refresh_token,
            'redirect_uri': 'http://localhost:8888/callback',
        }

        r_refresh = requests.post(TOKEN_URL, headers=headers, params=pars_refresh)
        access_token = r_refresh.json()['access_token']

        headers = {
        'Authorization': f'Bearer {access_token}'
                    }

        r_play = requests.get(play_url, headers=headers)
        
    r_play = r_play.json()
    # print(r_play['items'][0])
    df_items = df_items.append(r_play['items'], ignore_index=True)
    # print(df_items.head())
    offset += 50



In [ ]:
df_pub = df_items[df_items['public'] == True]

In [ ]:
df_list = []

for p_id in df_pub['id']:

    df_tracks = pd.DataFrame()

    offset = 0

    while len(df_tracks) % 100 == 0:
        
        # print(len(df_tracks))

        track_url = f'https://api.spotify.com/v1/playlists/{p_id}/tracks?limit=100&offset={offset}'
        headers = {
            'Authorization': f'Bearer {access_token}'
        }

        r_track = requests.get(track_url, headers=headers)

        track_ids = [t['track']['id'] for t in r_track.json()['items']]
        track_names = [t['track']['name'] for t in r_track.json()['items']]

        df_t = pd.DataFrame({
                'name': track_names,
                'id': track_ids,
                'playlist id': p_id

        })  

        join_ids = ','.join(track_ids)
        feat_url = f'https://api.spotify.com/v1/audio-features?limit=100&offset={offset}&ids={join_ids}'
        r_feat = requests.get(feat_url, headers=headers)
        feat_frame = pd.DataFrame(r_feat.json()['audio_features'])

        df_t = pd.merge(df_t, feat_frame, on='id')

        df_tracks = df_tracks.append(df_t)
        offset += 100

    df_list.append(df_tracks)

df_tracks = pd.concat(df_list)

In [ ]:
id_map = df_pub[['id', 'name']].set_index('id').to_dict()['name']
df_tracks['playlist id'] = df_tracks['playlist id'].map(id_map)

In [ ]:
df_tracks.head()

,name,id,playlist id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,Thunder,0afGLa6wcYiLLXFZ48ZQpY,søndag,0.667,0.316,0,-9.132,1,0.0262,0.0292,0.006130,0.147,0.156,100.998,audio_features,spotify:track:0afGLa6wcYiLLXFZ48ZQpY,https://api.spotify.com/v1/tracks/0afGLa6wcYiL...,https://api.spotify.com/v1/audio-analysis/0afG...,313663,4
1,Bedroom,3LH6EfPCeySeuymuK2gWAO,søndag,0.692,0.452,6,-8.180,1,0.1160,0.0358,0.000000,0.157,0.623,184.876,audio_features,spotify:track:3LH6EfPCeySeuymuK2gWAO,https://api.spotify.com/v1/tracks/3LH6EfPCeySe...,https://api.spotify.com/v1/audio-analysis/3LH6...,186781,4
2,Cherry Blossom,0bonUwkDjTgQ8URsgCcHhH,søndag,0.805,0.176,7,-20.783,1,0.0617,0.1020,0.000005,0.200,0.699,124.013,audio_features,spotify:track:0bonUwkDjTgQ8URsgCcHhH,https://api.spotify.com/v1/tracks/0bonUwkDjTgQ...,https://api.spotify.com/v1/audio-analysis/0bon...,198024,4
3,Pink & Blue (RAC Mix),3rxYhGVE6GHQyAy4CdGfq1,søndag,0.671,0.721,5,-7.378,1,0.0384,0.1310,0.376000,0.199,0.448,118.043,audio_features,spotify:track:3rxYhGVE6GHQyAy4CdGfq1,https://api.spotify.com/v1/tracks/3rxYhGVE6GHQ...,https://api.spotify.com/v1/audio-analysis/3rxY...,217271,4
4,Redstripe Rhapsody,2H7Nwzydg8ZusjdWkYqsHy,søndag,0.633,0.326,4,-15.050,0,0.3260,0.5020,0.001640,0.115,0.584,120.043,audio_features,spotify:track:2H7Nwzydg8ZusjdWkYqsHy,https://api.spotify.com/v1/tracks/2H7Nwzydg8Zu...,https://api.spotify.com/v1/audio-analysis/2H7N...,480000,4
